# MCMC CALIBRATION TECHNICS IN CONTEXT OF  INFECTIOUS DISEASE MODELING

## Prerequies

In [ ]:
#pip install summerepi2 estival jax==0.4.24 jaxlib==0.4.24 nevergrad jupyter pymc numpyro

In [3]:
# This is required for pymc parallel evaluation in notebooks

import multiprocess as mp
import platform

if platform.system() != "Windows":
    
    mp.set_start_method('forkserver')

In [4]:
import pandas as pd
from scipy import stats
import numpy as np
import plotly.express as px
pd.options.plotting.backend = "plotly"
import summer2
from summer2 import CompartmentalModel
from summer2.parameters import Parameter


# The following imports are the 'building blocks' of estival models

# Targets represent data we are trying to fit to
from estival import targets as est
import estival

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
import estival.model
from estival.model import BayesianCompartmentalModel


WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Calibration Data

In [ ]:
# An example of data for the calibration 

# output_labels = {"index": "time", "value": "number infectious"}
# data = pd.DataFrame(
#     {"active_cases":
#      {
#          60.0: 3000.0,
#          80.0: 8500.0,
#          100.0: 21000.0,
#          120.0: 40000.0,
#          140.0: 44000.0,
#          160.0: 30000.0,
#          180.0: 16000.0,
#          200.0: 7000.0,
#      },
#     },
# )
# data["active_cases"].plot(kind="scatter", labels=output_labels)

## Model Definition

A mechanistic model (ODE-Based) model discribing Infectious Disease transmission.

In [5]:
def get_sir_model(
    config: dict,
) -> CompartmentalModel:

    model = CompartmentalModel(
        times=(0.0, config["end_time"]),
        compartments=(
            "susceptible", 
            "infectious", 
            "recovered",
        ),
        infectious_compartments=("infectious",),
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        },
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery",
        fractional_rate=Parameter("recovery"),
        source="infectious",
        dest="recovered",
    )
    
    return model